In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import torch
import sys
sys.path.append('/Users/mehdi/github/sysnetdev')
from sysnet.sources.models import DNN
from sysnet.sources.io import load_checkpoint

In [ ]:
def load_l0weights(pid):
    """ load model """ 
    model = DNN(*(5, 20, 18, 1))
    
    path = '../output/mock001_cp2p_adamw/model/'
    load_checkpoint(f'{path}model_{pid}_2664485226/best.pth.tar', model)
    
    fc0_weight = model.fc[0].weight.data.numpy()
    return fc0_weight

In [ ]:
def imshow(f0w):
    fig, ax = plt.subplots()
    
    ylabels = ['EBV', 'lnHI', 'nstar']\
            + ['-'.join([s, b]) for s in ['depth', 'seeing', 'skymag', 'exptime', 'mjd'] \
               for b in 'rgz']    
    
    map1 = ax.imshow(f0w.T, origin='lower', cmap=plt.cm.bwr, vmin=-.5, vmax=.5)#, vmin=-0.3, vmax=0.3)
    fig.colorbar(map1)
    ax.set_yticks(np.arange(18))
    ax.set_yticklabels(ylabels)
    
    return ax

In [ ]:
f0w = []
for pid in range(5):
    f0w.append(load_l0weights(pid))

In [ ]:
%matplotlib inline

In [ ]:
for f0wi in f0w:
    imshow(f0wi)

In [ ]:
xlabels = ['EBV', 'lnHI', 'nstar']\
        + ['-'.join([s, b]) for s in ['depth', 'seeing', 'skymag', 'exptime', 'mjd'] \
           for b in 'rgz']    

for f0wi in f0w:    
    plt.scatter(np.arange(18), abs(f0wi.mean(axis=0)), alpha=0.4)
    
plt.ylabel('|weight_i|')
plt.ylim(ymax=0.26)
_=plt.xticks(np.arange(18), labels=xlabels, rotation=90)

In [ ]:
xlabels = ['EBV', 'lnHI', 'nstar']\
        + ['-'.join([s, b]) for s in ['depth', 'seeing', 'skymag', 'exptime', 'mjd'] \
           for b in 'rgz']    

for f0wi in f0w:    
    plt.scatter(np.arange(18), abs(f0wi.mean(axis=0)), alpha=0.4)
    
plt.ylabel('|weight_i|')
_=plt.xticks(np.arange(18), labels=xlabels, rotation=90)

In [ ]:
def imshow2(fc0w):
    ylabels = ['EBV', 'lnHI', 'nstar']\
            + ['-'.join([s, b]) for s in ['depth', 'seeing', 'skymag', 'exptime', 'mjd'] \
               for b in 'rgz']        
    
    plt.figure(figsize=(6, 10))
    
    plt.imshow(abs(fc0w.mean(axis=0)[:, np.newaxis]), 
               cmap=plt.cm.Blues, extent=(0, 5, -0.5, 17.5), 
               origin='lower')
    
    plt.yticks(np.arange(18), labels=ylabels)
    
    plt.xticks([])
    plt.colorbar()    

In [ ]:
for f0wi in f0w:
    imshow2(f0wi)

## correlation 

In [ ]:
df = np.load('../input/001/cp2p/cp2p_001.hp.256.5.r.npy', allow_pickle=True).item()

In [ ]:
df.keys()

In [ ]:
dt = np.concatenate([df['test']['fold%d'%i] for i in range(5)])

In [ ]:
dt

In [ ]:
from scipy.stats import pearsonr

In [ ]:
f(x, y) = 0.3 x + 0.7 y  # x and y are correlated, e.g., cov (x, y) ~ 1
f(x, y) = 0.7 x + 0.3 y

In [ ]:
fig, ax = plt.sub
